# Retrieve & Re-Ranking LAB

Podstawą wszystkich rozwiązań typu RAG (ang. Retrieval Augmented Generation) jest komponent wyszukiwania. Wyszukiwanie polega na identyfikacji odpowiednich dokumentów, w naszym RAGOwym przypadku takich dokumentów, które potencjalnie zawierają informacje kluczowe do odpowiedzenia na postawione pytanie. Definicja dokumentu jest też rzeczą płynną, raz to może być paragraf, a raz strona, czy rozdział, zależy to od zadania, oraz modeli jakie używamy.

Na wykładzie starałem się wyraźnie zwrócić uwagę na to, że wyszukiwanie może być leksykalne (BM25) lub semantyczne (najczęściej oparte na głębokich sieciach neuronowych, używanych jako modeli embedderów tekstów lub ocaniejących dopasowanie pary tekst-pytanie).
Podczas wykładu dokładnie omówiliśmy dwie kluczowe architektury głębokich sieci Transformer stosowanych do wyszukiwania informacji - bi-encoder i cross-encoder.

Przykładem bi-encodera jest `SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')`


```
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

docs = [
    "My first paragraph. That contains information",
    "Python is a programming language.",
]
document_embeddings = model.encode(docs)

query = "What is Python?"
query_embedding = model.encode(query)

similarity_scores = embedder.similarity(query_embedding, document_embeddings)

```


Przykładem CrossEncoder (`CrossEncoder('cross-encoder/ms-marco-MiniLM-L6-v2')`), który w odróżnieniu od bi-encodera ocenia odpowiedniość dokumentu do zapytania, zwracając dla podanej pary tekstów ich wskaźnik dopasowania (prawdopodobieństwo dopasowania).

```
from sentence_transformers import CrossEncoder
import torch

# Load https://huggingface.co/cross-encoder/ms-marco-MiniLM-L6-v2
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2", activation_fn=torch.nn.Sigmoid())
scores = model.predict([
    ("How many people live in Berlin?", "Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers."),
    ("How many people live in Berlin?", "Berlin is well known for its museums."),
])
```

Do pracy wymagamy zainstalowania trzech pakietów - `sentence-transformers` oraz `rank_bm25` oraz `datasets`


In [ ]:
!pip install -U sentence-transformers rank_bm25 datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

Zanim przejdziemy do zdefiniowania zadania przygotujemy sobie kilka przykładów użycia różnych metod i modeli, abyście mieli podstawy do realizacji swojego wyzwania.


Przykładowe użycie leksykalnego wyszukiwania opartego na BM25

In [ ]:
from rank_bm25 import BM25Okapi

# Korpus z przykładowymi tekstami
corpus = [
    "Trzeba jeść dużo warzyw, owoców i uprawiać sport.",
    "Trzeba pić alkohol, imprezować i palić.",
    "Kampania polityczna bardzo napawa optymizmem, same mądre głowy chcą rządzić Polakami.",
    "Historia Polski to wiele wieków walki o niepodległość.",
    "Przemysł motoryzacyjny od lat walczy z rosnącymi kosztami produkcji.",
]

tokenized_corpus = [doc.lower().split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

## KIEDY ZAPYTANIE MA LEKSYKALNE PRZECIĘCIE z KORPUSEM

query_0 = "O czym jest historia Polski?"
print(query_0)

tokenized_query_0 = query_0.lower().split(" ")

#policz dopasowanie leksykalne zapytania do każdego dokumentu w korpusie
doc_scores_0 = bm25.get_scores(tokenized_query_0)
print(doc_scores_0)

#znajdz top N najbliższych dokumentów do zadanego zapytania
print(bm25.get_top_n(tokenized_query_0, corpus, n=1))

## KIEDY ZAPYTANIE NIE MA LEKSYKALNEGO PRZECIĘCIE z KORPUSEM

query_1 = "Jak długo żyć?"
print(query_1)

tokenized_query_1 = query_1.lower().split(" ")

#policz dopasowanie leksykalne zapytania do każdego dokumentu w korpusie
doc_scores_1 = bm25.get_scores(tokenized_query_1)
print(doc_scores_1)

#znajdz top N najbliższych dokumentów do zadanego zapytania
print(bm25.get_top_n(tokenized_query_1, corpus, n=1))



O czym jest historia Polski?
[0.         0.         0.         2.24533898 0.        ]
['Historia Polski to wiele wieków walki o niepodległość.']
Jak długo żyć?
[0. 0. 0. 0. 0.]
['Przemysł motoryzacyjny od lat walczy z rosnącymi kosztami produkcji.']


Przykładowe użycie semantycznego wyszukiwania bazującego na rozwiazaniach typu bi-encoder dla j.polskiego.

In [ ]:
import torch

from sentence_transformers import SentenceTransformer


embedder = SentenceTransformer("sdadas/mmlw-retrieval-roberta-large")

query_prefix = "zapytanie: "
answer_prefix = ""

# Korpus z przykładowymi tekstami
corpus = [
    answer_prefix + "Trzeba jeść dużo warzyw, owoców i uprawiać sport.",
    answer_prefix + "Trzeba pić alkohol, imprezować i palić.",
    answer_prefix + "Kampania polityczna bardzo napawa optymizmem, same mądre głowy chcą rządzić Polakami.",
    answer_prefix + "Historia Polski to wiele wieków walki o niepodległość.",
    answer_prefix + "Przemysł motoryzacyjny od lat walczy z rosnącymi kosztami produkcji.",
]



# Use "convert_to_tensor=True" to keep the tensors on GPU (if available)
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Zapytania
queries = [
    query_prefix + "Jak dożyć 100 lat?",
    query_prefix + "Z czym walczy sektor automoto?"
]

# Znajdz dwa najbliższe dokumenty z korpusu dla zadanego zapytania używajać podobieństwa opartego na mierze cosinus
top_k = min(2, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 2 scores
    similarity_scores = embedder.similarity(query_embedding, corpus_embeddings)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)

    print("\nZapytanie:", query)
    print("Top 2 najbliższych dokumentów:")

    for score, idx in zip(scores, indices):
        print(corpus[idx], f"(Score: {score:.4f})")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/8.59M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]


Zapytanie: zapytanie: Jak dożyć 100 lat?
Top 2 najbliższych dokumentów:
Trzeba jeść dużo warzyw, owoców i uprawiać sport. (Score: 0.7190)
Trzeba pić alkohol, imprezować i palić. (Score: 0.6929)

Zapytanie: zapytanie: Z czym walczy sektor automoto?
Top 2 najbliższych dokumentów:
Przemysł motoryzacyjny od lat walczy z rosnącymi kosztami produkcji. (Score: 0.7719)
Historia Polski to wiele wieków walki o niepodległość. (Score: 0.6726)


Przykładowe użycie semantycznego wyszukiwania w zakresie komponentu re-rankera opartego na polskich cross-encoderach.

In [ ]:
from sentence_transformers import CrossEncoder
import torch.nn

query = "Jak dożyć spokojnej starości w dobrej kondycji?"
answers = [
    "Trzeba zdrowo się odżywiać, oraz unikać alkoholu i papierosów",
    "Trzeba pić alkohol, imprezować i chwytać dzień",
    "Życie to tylko ciąg wspomnień i obrazy z przeszłości."
]

model = CrossEncoder(
    "sdadas/polish-reranker-large-ranknet",
    device="cuda" if torch.cuda.is_available() else "cpu"
)
pairs = [[query, answer] for answer in answers]
results = model.predict(pairs)
print(results.tolist())


config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.59M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

[0.8108768463134766, 0.3784073293209076, 0.0719650462269783]


ZADANIE #1 PREPROCESSING - przetwórz korpus polskiej wikipedii do postaci kolekcji tekstów

In [ ]:
import json
import gzip
import os
import torch
from tqdm import tqdm

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

# Użyj dumpa polskiej Wikipedii.

wikipedia_hf_path = 'chrisociepa/wikipedia-pl-20230401'


passages = []

from datasets import load_dataset

dataset = load_dataset(wikipedia_hf_path, split="train", streaming=True)

print(next(iter(dataset)))

def chunk_text(text, chunk_size=200, overlap=50):
    tokens = text.split()
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = " ".join(tokens[i:i + chunk_size])
        if len(chunk.split()) >= 50:
            chunks.append(chunk)
    return chunks

passages_set = set()
max_articles = 200_000

for i, article in tqdm(enumerate(dataset), total=max_articles):
    if i >= max_articles:
        break
    text = article.get("text", "")
    if text:
        chunks = chunk_text(text)
        passages_set.update(chunks)

passages = list(passages_set)
print("Przykładowy pasaż:\n", passages[0])

##Napisz funkcję, która zbuduje min kilka set tys zbiór dokumentów zwanych pasażami
##Używając tej funkcji wypełnij zmienną passages, sprawdz i usun duplikaty

print("Passages:", len(passages))


README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

{'id': '2', 'url': 'https://pl.wikipedia.org/wiki/AWK', 'title': 'AWK', 'text': 'AWK – interpretowany język programowania, którego główną funkcją jest wyszukiwanie i przetwarzanie wzorców w plikach lub strumieniach danych. Jest także nazwą programu początkowo dostępnego dla systemów operacyjnych będących pochodnymi UNIX-a, obecnie także na inne platformy.\n\nAWK jest językiem, który w znacznym stopniu wykorzystuje tablice asocjacyjne, stringi i wyrażenia regularne. Nazwa języka pochodzi od pierwszych liter nazwisk jego autorów Alfreda V. Aho, Petera Weinbergera i Briana Kernighana. Bywa zapisywana małymi literami, odczytywana jako jedno słowo awk, wymawiana jak pierwsza sylaba w awkward.\n\nDefinicja języka AWK jest zawarta w POSIX 1003.2 Command Language And Utilities Standard. Wersja ta jest z kolei oparta na opisie z The AWK Programming Language napisanym przez Aho, Weinbergera i Kernighana, z dodatkowymi właściwościami zdefiniowanymi w wersji awk z SVR4.\n\nW wierszu poleceń podaje

100%|██████████| 200000/200000 [00:59<00:00, 3383.16it/s]


Przykładowy pasaż:
 istnieje wygrana i przegrana, czego brakuje np. wtedy, gdy sami odbijamy piłkę od ściany, dla sporej liczby gier liczy się zręczność, której brakuje np. w szachach – ale wszystkie one w jakiś sposób są ze sobą spokrewnione. Widzimy skomplikowaną siatkę zachodzących na siebie i krzyżujących się podobieństw; podobieństw w skali dużej i małej. (Wittgenstein) Metoda przykładów paradygmatycznych Wittgensteina Krok pierwszy: Celem określenia znaczenia nazwy układa się listę typowych przykładów (tzw. paradygmatów desygnatów), które na mocy przyjętych konwencji kulturowych podpadają pod jej zakres. Krok drugi: Listę taką można modyfikować na bazie nowych informacji i konwencji. Modyfikacja polega na rozszerzeniu o nowe przypadki oraz na redukcji przypadków niewłaściwych. To, który przypadek jest niewłaściwy, okazuje się przy bliższym sprawdzeniu – określeniu podobieństwa rodzinnego. Z biegiem czasu osiągnięta zostaje lista minimalna, czyli taka, której zredukować nie można,

ZADANIE #2 PRZYGOTOWANIE KOMPONENTÓW - BI-ENCODERA, CROSS-ENCODERA i BM25

In [ ]:

#inicjalizacja Bi-Encodera, dokładniej enkodera używanego w obu gałęziach, aby zbudować embeddingi dla tekstów i zapytania
#sam wybierz dokładny model pod kątem j.polskiego
bi_encoder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

#inicjalizacja Cross-encoder do  re-rankowania wyników
#sam wybierz dokładny model pod kątem j.polskiego
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# lexical search (keyword search) z użyciem BM25 metody

from rank_bm25 import BM25Okapi
import string
from tqdm import tqdm

polish_stopwords = {
    'i', 'oraz', 'a', 'ale', 'więc', 'lecz', 'czy', 'bo', 'gdy', 'gdyż',
    'dla', 'do', 'na', 'z', 'za', 'po', 'pod', 'przed', 'bez', 'o', 'od',
    'jak', 'że', 'który', 'która', 'którzy', 'które', 'ten', 'ta', 'to',
    'by', 'był', 'była', 'było', 'jest', 'są', 'nie', 'czyli', 'też', 'jeśli'
}

# Tokenizacja pod BM25 z uwzglednieniem lower case,  usuwaniem stop-words
def bm25_tokenizer(text):
    tokenized_doc = []
    #uzupełnij kod
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    tokenized_doc = text.split()
    tokenized_doc = [token for token in tokenized_doc if token not in polish_stopwords]
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(passages):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)



100%|██████████| 357665/357665 [01:08<00:00, 5255.18it/s]


Batches:   0%|          | 0/11178 [00:00<?, ?it/s]

ZADANIE #3 - FINALNE ROZWIAZANIE - napisz funkcje, która będzie używała BM25 i BI_ENCODERA do wyszukiwania, potem wyniki z obu tych modułów wyszukiwania należy zmergować i dokonać ich re-rankingu i zwrócić top 10 wyników.

In [ ]:
# Funkcja, która dla tekstowego zapytania wykona Hybrid Search (BM25, BI-ENCODER), potem dokona RE-RANKINGU i zwróci top 10
from sentence_transformers.util import cos_sim

def search(query, top_n=10, bm25_top_k=50, bi_encoder_top_k=50):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    query_tokens = bm25_tokenizer(query)
    bm25_results = bm25.get_top_n(query_tokens, passages, n=bm25_top_k)
    print(f"BM25 retrieved {len(bm25_results)} passages")

    ##### Semantic Search using BI-ENCODER #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    cosine_scores = cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cosine_scores, bi_encoder_top_k)
    bi_encoder_results = [passages[idx] for idx in top_results.indices]
    print(f"Bi-Encoder retrieved {len(bi_encoder_results)} passages")

    #### MERGE RESULTS FROM BM25 and BI-ENCODER####
    merged_passages = list(set(bm25_results + bi_encoder_results))
    print(f"Merged total unique passages: {len(merged_passages)}")

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [(query, passage) for passage in merged_passages]
    scores = cross_encoder.predict(cross_inp)

    ranked_results = sorted(zip(merged_passages, scores), key=lambda x: x[1], reverse=True)

    # RETRIEVE TOP N
    print(f"Top {top_n} results:")
    for i, (passage, score) in enumerate(ranked_results[:top_n]):
        print(f"{i+1:02d}. [Score: {score:.4f}] {passage[:200]}...")


In [ ]:
search(query = "Ile mieszkańców ma Warszawa?")

Input question: Ile mieszkańców ma Warszawa?
BM25 retrieved 50 passages
Bi-Encoder retrieved 50 passages
Merged total unique passages: 100
Top 10 results:
01. [Score: 2.4316] Ericha von dem Bach-Zelewskiego rozkaz brzmiał następująco: „każdego mieszkańca należy zabić, nie wolno brać żadnych jeńców. Warszawa ma być zrównana z ziemią i w ten sposób ma być stworzony zastrasza...
02. [Score: 1.2536] Łąkoć (dawn. Ląkoć) – wieś w Polsce położona w województwie lubelskim, w powiecie puławskim, w gminie Kurów. Liczy 231 mieszkańców i ma powierzchnię 5,8 km². W latach 1975–1998 miejscowość administrac...
03. [Score: 0.6495] do pierwszoligowego zespołu Górnika Wałbrzych. 1 czerwca 2020 został zawodnikiem Legii Warszawa. Osiągnięcia Stan na 14 sierpnia 2022, na podstawie, o ile nie zaznaczono inaczej. Drużynowe Seniorskie ...
04. [Score: 0.5876] W latach 2002-2017 liczba mieszkańców zmalała o 21,3%. Średni wiek mieszkańców wynosi 42,2 lat i jest nieznacznie większy od średniego wieku mieszkańców 

In [ ]:
search(query = "Jak długa jest Wisła?")

Input question: Jak długa jest Wisła?
BM25 retrieved 50 passages
Bi-Encoder retrieved 50 passages
Merged total unique passages: 100
Top 10 results:
01. [Score: 5.3289] Przednutka długa () – ozdobnik, w którym przed głównym dźwiękiem dodawany jest dodatkowy o dowolny interwał (często sekundę) wyższy lub niższy od niego. Przednutka długa skraca nutę ozdabianą o wartoś...
02. [Score: 4.3515] Riasa (od gr. ρασα lub ραχος) – jest to długa, z reguły czarna, opuszczona, nie ściągnięta w pasie szata. Jej nazwa pochodzi od greckiego czasownika ρασσω - rozpraszać, marszczyć. Jest to długa, sięga...
03. [Score: 4.0132] ostoją polskości w pierwszych dziesięcioleciach XX wieku w Bydgoszczy. W 2017 elewację budynku ponownie poddano remontowi. Architektura Budynek jest wzniesiony na rzucie podkowy. Kamienica frontowa je...
04. [Score: 4.0000] ją do miasta Sulejówek: Przemysł Elektroniczny: siedziba firmy Media-Tech Polska w Brzezinach Zakład produkcyjny Spółdzielni Inwalidów Świt w Halinowie Kultura 

In [ ]:
search(query = "Podaj pięciu polskich Noblistów?")

Input question: Podaj pięciu polskich Noblistów?
BM25 retrieved 50 passages
Bi-Encoder retrieved 50 passages
Merged total unique passages: 100
Top 10 results:
01. [Score: 3.5670] Szkoła Podstawowa nr 367 im. Polskich Noblistów . W 1962 utwardzono ulicę Józefa Mehoffera od przejazdu kolejowo-drogowego do ulicy Raciborskiej i do Choszczówki dotarła pierwsza w historii linia auto...
02. [Score: 2.0341] świętokrzyska GKS Gród Ćmińsk w sezonie 2008/2009 – klasa "B" Edukacja Na terenie gminy Miedziana Góra funkcjonują trzy szkoły podstawowe i dwa gimnazja. Zespół Szkół w Ćmińsku Szkoła Podstawowa w Ćmi...
03. [Score: 1.9953] Schronisko PTTK w Dolinie Pięciu Stawów Polskich (Schronisko Pięciostawiańskie) – schronisko turystyczne położone w Dolinie Pięciu Stawów Polskich w Tatrach Wysokich. Zostało ono zbudowane w latach 19...
04. [Score: 1.9652] Turystyka Przez miasto przebiega Wielkopolska Droga św. Jakuba i Dolnośląska Droga św. Jakuba – odcinek szlaku pielgrzymkowego do grobu św. Jakuba w 

In [ ]:
search(query = "Jak nazywał się pierwszy król Polski?")

Input question: Jak nazywał się pierwszy król Polski?
BM25 retrieved 50 passages
Bi-Encoder retrieved 50 passages
Merged total unique passages: 99
Top 10 results:
01. [Score: 6.8805] Król Polski (łac. rex Poloniae) – tytuł monarszy koronowanych władców Królestwa Polskiego, Korony Królestwa Polskiego i Rzeczypospolitej. Historia Pierwszym królem Polski był Bolesław I Chrobry, koron...
02. [Score: 6.2778] wojskami królewskimi. 1520 – Wojna pruska: Krzyżacy zdobyli Braniewo. 1659 – Potop szwedzki: król Jan II Kazimierz wjechał triumfalnie do odzyskanego Torunia. 1791 – W Warszawie ukazało się pierwsze w...
03. [Score: 6.1550] August został koronowany vivente rege na króla Polski. 1532 – Podpisano rozejm polsko-mołdawski kończący wojnę o Pokucie. 1578 – Przed kościołem św. Anny w Warszawie margrabia Jerzy Fryderyk Hohenzoll...
04. [Score: 6.1493] Węgierski został królem Polski. 1847 – W Berlinie zakończył się proces wielkopolskich powstańców i działaczy niepodległościowych. Skazano 117 osó